In [ ]:
import time
from multiprocessing import Pool
import glob
import os
from natsort import natsorted
from postprocessing import *

In [ ]:
mpl.rcParams['figure.dpi']= 200
# mpl.rcParams['text.usetex'] = True
# mpl.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}']

In [ ]:
density = '20'
mach    = '14'
jet     = '40'
start   = '15'

simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jet}_{start}')
#simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/turb')
#simulation_directory = str(f'/n/holystore01/LABS/hernquist_lab/Users/borodina/2kpc/turb_drive_center_d{density}_m{mach}/turb_alter_{start}')

output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

bins = np.linspace(-900, 900, 91)
bins_step = bins[1] - bins[0]

In [ ]:
i_file = 12 - 1
while True:
    i_file += 1

    projection = 'radial'
    try:
        filename_jet = "snap_%03d.hdf5" % (i_file)
        snap_data_jet = h5py.File(output_directory + filename_jet, "r")
    except:
        break
    
    x, y, z = snap_data_jet['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data_jet['PartType0/Velocities'][:].T
    center = snap_data_jet['Header'].attrs['BoxSize'] / 2
    v_r_jet = (vx * (x - center) + vy * (y - center) + vz * (z - center)) / \
                np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)

    temperatures_jet = get_temp(output_directory + filename_jet, 5/3)
    masses_jet = snap_data_jet['PartType0/Masses'][:]
    densities_jet = snap_data_jet['PartType0/Density'][:]
    jettracer = snap_data_jet['PartType0/Jet_Tracer'][:]
    volumes_jet = masses_jet / densities_jet 
    radius_jet = np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)

    mask_hot_jet  = ((temperatures_jet > 10 ** 4.5) & (radius_jet > 400) & (radius_jet < 500))
    mask_warm_jet = ((temperatures_jet < 10 ** 4.5) & (radius_jet > 400) & (radius_jet < 500))


    i_file_turb = 12
    projection = 'radial'

    filename_turb = "snap_%03d.hdf5" % (i_file_turb)
    snap_data_turb = h5py.File(output_directory + filename_turb, "r")

    x, y, z = snap_data_turb['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data_turb['PartType0/Velocities'][:].T
    v_r_turb = (vx * (x - center) + vy * (y - center) + vz * (z - center)) / np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)

    temperatures_turb = get_temp(output_directory + filename_turb, 5/3)
    masses_turb = snap_data_turb['PartType0/Masses'][:]
    densities_turb = snap_data_turb['PartType0/Density'][:]
    volumes_turb = masses_turb / densities_turb 
    radius_turb = np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)

    mask_hot_turb  = ((temperatures_turb > 10 ** 4.5) & (radius_turb > 400) & (radius_turb < 500))
    mask_warm_turb = ((temperatures_turb < 10 ** 4.5) & (radius_turb > 400) & (radius_turb < 500))

    fig, ax = plt.subplots(figsize=(5.5, 3))
    ax.hist((v_r_turb[mask_hot_turb]), bins=bins, 
             log=True, weights=masses_turb[mask_hot_turb] / bins_step, label='hot gas in initial box', alpha=0.5, color='tab:red')
    ax.hist((v_r_turb[mask_warm_turb]), bins=bins, 
             log=True, weights=masses_turb[mask_warm_turb] / bins_step, label='warm gas in initial box', alpha=0.5, color='tab:blue')


    ax.hist((v_r_jet[mask_hot_jet]), bins=bins, histtype='step', 
             log=True, weights=masses_jet[mask_hot_jet] / bins_step, label='hot gas when jet is on', ec='tab:red',linewidth=1.5)
    ax.hist((v_r_jet[mask_warm_jet]), bins=bins, histtype='step', 
             log=True, weights=masses_jet[mask_warm_jet] / bins_step, label='warm gas when jet is on', ec='tab:blue', linewidth=1.5)


    plt.title(fr't = {np.round(get_time_from_snap(snap_data_jet) * unit_time_in_megayr, 2)} Myr')
    plt.legend(loc='upper right',fontsize=7, fancybox=True, framealpha=0.5)
    plt.grid(ls='--', zorder = 100, alpha=0.5)
    ax.set_xlim(-800, 800)
    ax.set_ylim(1e-1, 5e6)
    ax.set_xlabel(r"$v_r$ [km / s]")
    ax.set_ylabel(r'$\frac{M_\odot}{\text{km / s}}$ of cells with $r_\text{cell} \in [400; 500]$ pc')
#     ax.set_facecolor('w')
#     fig.set_facecolor('none')

#     params = {"ytick.color" : "w",
#           "xtick.color" : "w",
#           "axes.labelcolor" : "w",
#           "axes.edgecolor" : "w",
#           "text.color" : "w",
#           "figure.facecolor" : '2F366E'}
#     plt.rcParams.update(params)
    plt.legend(loc='upper right',fontsize=7, fancybox=True, framealpha=0.5, labelcolor='black')
    
    plt.savefig(figures_directory + f'velocity_distribution_{i_file}.png', dpi=400, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + '/velocity_distribution*.png'
ofilename = figures_directory + '/velocity_distribution.gif'
imgs = natsorted(glob.glob(ifilename))

timestep=4

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=len(imgs) * timestep, loop=0)

In [ ]:
a = np.sort(v_r_jet[mask_hot_jet])
a[-300:-200]

## averaging histogram

In [ ]:
density = '20'
mach    = '14'
jet     = '40'
start   = '15'

simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jet}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

bins = np.linspace(-900, 900, 91)
bins_step = bins[1] - bins[0]

In [ ]:
t_start_40 = 16.5
t_start_38 = 17.6
t_start_43 = 15
t_start_4  = 17.5
t_start_14 = 17.5

t_start = t_start_14

In [ ]:
velocities_hot = []
velocities_warm = []
i_file = 12 - 1
while True:
    i_file += 1

    projection = 'radial'
    try:
        filename_jet = "snap_%03d.hdf5" % (i_file)
        snap_data_jet = h5py.File(output_directory + filename_jet, "r")
        time = get_time_from_snap(snap_data_jet) * unit_time_in_megayr
        if time < t_start:
            continue
        if time > t_start + 1:
            break
    except:
        break
    
    x, y, z = snap_data_jet['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data_jet['PartType0/Velocities'][:].T
    center = snap_data_jet['Header'].attrs['BoxSize'] / 2
    v_r_jet = (vx * (x - center) + vy * (y - center) + vz * (z - center)) / \
                np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)

    temperatures_jet = get_temp(output_directory + filename_jet, 5/3)
    masses_jet = snap_data_jet['PartType0/Masses'][:]
    densities_jet = snap_data_jet['PartType0/Density'][:]
    jettracer = snap_data_jet['PartType0/Jet_Tracer'][:]
    volumes_jet = masses_jet / densities_jet 
    radius_jet = np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)

    mask_hot_jet  = ((temperatures_jet > 10 ** 4.5) & (radius_jet > 400) & (radius_jet < 500))
    mask_warm_jet = ((temperatures_jet < 10 ** 4.5) & (radius_jet > 400) & (radius_jet < 500))

    velocities_hot.append(np.histogram((v_r_jet[mask_hot_jet]), bins=bins, weights=masses_jet[mask_hot_jet] / bins_step)[0])
    velocities_warm.append(np.histogram((v_r_jet[mask_warm_jet]), bins=bins, weights=masses_jet[mask_warm_jet] / bins_step)[0])
    

In [ ]:
velocities_average_hot = np.mean(velocities_hot, axis=0)
velocities_average_warm = np.mean(velocities_warm, axis=0)

In [ ]:
velocities_hot_turb = []
velocities_warm_turb = []
simulation_directory_turb = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/turb')
output_directory_turb = simulation_directory_turb + "/output/"

i_file_turb = 12 - 1
while True:
    i_file_turb += 1

    projection = 'radial'
    try:
        filename_turb = "snap_%03d.hdf5" % (i_file_turb)
        snap_data_turb = h5py.File(output_directory_turb + filename_turb, "r")
        time = get_time_from_snap(snap_data_turb) * unit_time_in_megayr
        if time < t_start:
            continue
        if time > t_start + 1:
            break
    except:
        break
    
    x, y, z = snap_data_turb['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data_turb['PartType0/Velocities'][:].T
    center = snap_data_turb['Header'].attrs['BoxSize'] / 2
    v_r_turb = (vx * (x - center) + vy * (y - center) + vz * (z - center)) / \
                np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)

    temperatures_turb = get_temp(output_directory_turb + filename_turb, 5/3)
    masses_turb = snap_data_turb['PartType0/Masses'][:]
    densities_turb = snap_data_turb['PartType0/Density'][:]
    jettracer = snap_data_turb['PartType0/Jet_Tracer'][:]
    volumes_turb = masses_turb / densities_turb 
    radius_turb = np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)

    mask_hot_turb  = ((temperatures_turb > 10 ** 4.5) & (radius_turb > 400) & (radius_turb < 500))
    mask_warm_turb = ((temperatures_turb < 10 ** 4.5) & (radius_turb > 400) & (radius_turb < 500))

    velocities_hot_turb.append(np.histogram((v_r_turb[mask_hot_turb]), bins=bins, weights=masses_turb[mask_hot_turb] / bins_step)[0])
    velocities_warm_turb.append(np.histogram((v_r_turb[mask_warm_turb]), bins=bins, weights=masses_turb[mask_warm_turb] / bins_step)[0])
    

In [ ]:
velocities_average_hot_turb = np.mean(velocities_hot_turb, axis=0)
velocities_average_warm_turb = np.mean(velocities_warm_turb, axis=0)

In [ ]:
i_file_turb = 12
projection = 'radial'

filename_turb = "snap_%03d.hdf5" % (i_file_turb)
snap_data_turb = h5py.File(output_directory + filename_turb, "r")

x, y, z = snap_data_turb['PartType0/Coordinates'][:].T
vx, vy, vz = snap_data_turb['PartType0/Velocities'][:].T
v_r_turb = (vx * (x - center) + vy * (y - center) + vz * (z - center)) / np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)

temperatures_turb = get_temp(output_directory + filename_turb, 5/3)
masses_turb = snap_data_turb['PartType0/Masses'][:]
densities_turb = snap_data_turb['PartType0/Density'][:]
volumes_turb = masses_turb / densities_turb 
radius_turb = np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)

mask_hot_turb  = ((temperatures_turb > 10 ** 4.5) & (radius_turb > 400) & (radius_turb < 500))
mask_warm_turb = ((temperatures_turb < 10 ** 4.5) & (radius_turb > 400) & (radius_turb < 500))

In [ ]:
fig, ax = plt.subplots(figsize=(5.5,3))


# ax.grid(ls='--', zorder = 100, alpha=0.5)
# ax.set_xlim(-800, 800)
# ax.set_ylim(1e-2, 1e6)
# ax.set_xlabel(r"$v_r$ [km / s]")

# ax.hist((v_r_jet_4[mask_hot_jet_4]), bins=bins, histtype='step', 
#          log=True, weights=masses_jet_4[mask_hot_jet_4] / bins_step, label='hot gas with jet on', ec='tab:red',linewidth=1.5)
# ax.hist((v_r_jet_4[mask_warm_jet_4]), bins=bins, histtype='step', 
#          log=True, weights=masses_jet_4[mask_warm_jet_4] / bins_step, label='warm gas with jet on', ec='tab:blue', linewidth=1.5)
ax.hist((v_r_turb[mask_warm_turb]), bins=bins, 
         log=True, weights=masses_turb[mask_warm_turb] / bins_step, label='warm gas', alpha=0.5, color='tab:blue', zorder=0)
ax.hist((v_r_turb[mask_hot_turb]), bins=bins, 
         log=True, weights=masses_turb[mask_hot_turb] / bins_step, label='hot gas', alpha=0.5, color='tab:red', zorder=0)

ax.step(bins[:-1], velocities_average_warm, where='mid', label='warm gas with jet on', color='tab:blue', linewidth=1.5)
ax.step(bins[:-1], velocities_average_hot, where='mid', label='hot gas with jet on', color='tab:red', linewidth=1.5)

# ax.fill_between(bins[:-1], velocities_average_warm_turb, step='mid', label='warm gas', color='tab:blue', alpha=0.5, edgecolor='none')
# ax.fill_between(bins[:-1], velocities_average_hot_turb, step='mid', label='hot gas', color='tab:red', alpha=0.5, edgecolor='none')


plt.title(fr'averaged over 1 Myr, M=8')
plt.legend(loc='upper right',fontsize=7, fancybox=True, framealpha=0.5)
plt.grid(ls='--', zorder = 100, alpha=0.5)
ax.set_xlim(-800, 800)
ax.set_ylim(1e-1, 5e6)
ax.set_yscale('log')
ax.set_xlabel(r"$v_r$ [km / s]")
ax.set_ylabel(r'$\frac{M_\odot}{{km / s}}$ of cells with $r_{cell} \in [400; 500]$ pc')
